<a href="https://colab.research.google.com/github/1eetnals/coursera-tfcertificate/blob/main/C2_W1_cat_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Click the button on top that says, `Open in Colab`, to run this notebook as a Colab. Running the notebook on your local machine might result in some of the code blocks throwing errors.

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [ ]:
# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset

# Note: This is a very large dataset and will take time to download
# wget을 이요해서 웹서버에 접근하는 경우 인증서가 유효하지 않는다는 이유로 실패 되는 경우가 있음
# 이런 경우 --no--check-certificate 옵션을 추가해서 인증서 체크하는 부분을 skip할 수 있음
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'  # colab tmp 폴더 안의 다운받은 파일 경로
zip_ref   = zipfile.ZipFile(local_zip, 'r')   #zipfile 라이브러리의 ZipFile 클래스로 ZIP 파일 열기
zip_ref.extractall('/tmp')  # extractall()을 이용해서 tmp 폴더에 압축 풀기
zip_ref.close()


--2022-01-27 06:16:49--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.201.212.130, 2600:1402:2000:1bb::e59, 2600:1402:2000:193::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.201.212.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M   109MB/s    in 6.3s    

2022-01-27 06:16:56 (125 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [ ]:
# 각 디렉토리에 몇 개의 강아지, 고양이 이미지가 존재하는지 출력
# os.listdir() method는 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 return 
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

12501
12501


In [ ]:
# Use os.mkdir to create your directories
# (os.mkdir을 사용해서 디렉토리 생성)
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'
# (cats-v-dogs를 위한 디렉토리와 training, testing을 위한 하위 디렉토리 필요. 그 다음으로는 cats, dogs를 위한 하위 디렉토리 필요.)
try:

  # directory 만들기
  ### START YOUR CODE HERE
  os.mkdir('/tmp/cats-v-dogs')  
  os.mkdir('/tmp/cats-v-dogs/training')
  os.mkdir('/tmp/cats-v-dogs/testing')
  os.mkdir('/tmp/cats-v-dogs/training/cats')
  os.mkdir('/tmp/cats-v-dogs/training/dogs')
  os.mkdir('/tmp/cats-v-dogs/testing/cats')
  os.mkdir('/tmp/cats-v-dogs/testing/dogs')


  ### alternate solution / 다른 표현 방법
  # classes = ['cats','dogs']
  
  # for class_name in classes:
  #     os.makedirs(os.path.join("/tmp/cats-v-dogs", "training", class_name))
  #     os.makedirs(os.path.join("/tmp/cats-v-dogs", "testing", class_name))

  ### END YOUR CODE HERE
except OSError:
    pass

생성된 디렉토리의 구조

*   cats-v-dogs
  *   training
      *   dogs 
      *   cats
  *   testing
      *   dogs
      *   cats


In [ ]:
# Write a python function called split_data which takes 
# (split_data라고 불리는 파이썬 함수를 작성하여 다음을 취함)
# a SOURCE directory containing the files
# (파일이 들어있는 SOURCE directory)
# a TRAINING directory that a portion of the files will be copied to
# (파일의 일부가 복사될 TRAINING directory)
# a TESTING directory that a portion of the files will be copie to
# (파일의 일부가 복사될 TESTING directory)
# a SPLIT SIZE to determine the portion
# (복사할 부분을 결정하는 SPLIT SIZE)
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# (training set가 파일의 X%가 랜덤이고 test set가 나머지 파일이 되도록 파일을 랜덤화해야함)
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# (예를 들어, SOURCE가 PetImages/Cat이고 SPLIT SIZE가 .9인 경우)
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# (# 그러면 PetImages/Cat의 90%의 이미지가 TRAINING dir에 복사되고 이미지의 10%가 TESTING dir로 복사됨)
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
# (또한 -- 모든 이미지를 검사해야하며 파일 길이가 0인 경우 복사되지 않음)
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# (os.listdir(Directory)는 해당 디렉토리의 내용 목록을 제공)
# os.path.getsize(PATH) gives you the size of the file
# (os.path.getsize(PATH)는 파일 크기를 제공)
# copyfile(source, destination) copies a file from source to destination
# (copyfile(source, destination)은 source에서 destination으로 파일을 복사)
# random.sample(list, len(list)) shuffles a list
# (random.sample(list, len(list))은 목록을 섞음)

# 이미지 복사하기
# Cat 폴더에 있던 고양이 이미지를 training/cats 디렉토리와 testing/cats 디렉토리에 나누어 저장
# Dog 폴더에 있던 강아지 이미지를 training/dogs 디렉토리와 testing/dogs 디렉토리에 나누어 저장
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

  ### START YOUR CODE HERE
  files = []  #파일을 담아줄 리스트 생성
  for filename in os.listdir(SOURCE):   #SOURCE 디렉토리에서 데이터들을 하나 하나 꺼냄
    file = SOURCE + filename 
    # SOURCE의 경로(/tmp/PetImages/Dog/)+ 파일 이름(dog.123) 형식으로 데이터들은 하나 하나 모임
    if os.path.getsize(file) > 0:
    # 위에서 나온 것처럼 사이즈가 0 초과일 때만 진행  
      files.append(filename)
      #통과한 데이터들 만들어놓은 file 리스트에 담기
    else:
      print(filename + " is zero length, so ignoring.")

  training_length = int(len(files) * SPLIT_SIZE)
  # 파일을 split_data 함수로 여러 주머니에다가 담아주어도, 파일의 총량은 변하지 않음
  # (주머니 안의 파일 갯수*나눠진 주머니의 갯수 = 총 파일의 갯수)
  testing_length = int(len(files) - training_length)
  shuffled_set = random.sample(files, len(files))
  training_set = shuffled_set[0:training_length]  # 섞는 범위 지정
  testing_set = shuffled_set[-testing_length:]

  for filename in training_set:
    this_file = SOURCE + filename
    # (/tmp/PetImages/Dog/)+(dog.123)의 형식을 this_file에 저장   
    destination = TRAINING + filename
    # 트레이닝 데이터들이 담긴 경로+(dog.123)의 형식으로 destination에 저장
    copyfile(this_file, destination)
    # copyfile 함수를 사용해 this_file을 destination에 복사
    
  for filename in testing_set:
    this_file = SOURCE + filename
    destination = TESTING + filename
    copyfile(this_file, destination)

  ### END CODE HERE

  ### alternate solution / 다른 표현 방법
  # # YOUR CODE STARTS HERE
  # files = os.listdir(SOURCE)
  # files = [file for file in files if os.path.getsize(os.path.join(SOURCE, file))>0]

  # files = random.sample(files, len(files))
  # limit = len(files) * SPLIT_SIZE

  # for index, file in enumerate(files):
  #   source = os.path.join(SOURCE, file)
  #   if index < limit:
  #     destination = os.path.join(TRAINING, file)
  #   else:
  #     destination = os.path.join(TESTING, file)  
    
  #   copyfile(source, destination)
  # # YOUR CODE ENDS HERE


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [ ]:
# train, test할 강아지, 고양이 이미지 수 확인
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

11250
11250
1250
1250


In [ ]:
# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
model = tf.keras.models.Sequential([
    ### START CODE HERE
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    # 강아지와 고양이를 분류하는 이진 분류 문제이기 때문에 마지막 layer의 node 수는 1개, activation funcion으로는 sigmoid 사용
    ### END CODE HERE
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

TRAINING_DIR = "/tmp/cats-v-dogs/training/" #YOUR CODE HERE
# ImageDataGenerator 생성 (rescale=1./255를 설정해 0~1 사이의 값으로 정규화)
train_datagen = ImageDataGenerator(rescale=1.0/255.) #YOUR CODE HERE
# train_datagen과 validation_datagen에서의 target_size는 무조건 동일해야 함
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, #YOUR CODE HERE
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/" #YOUR CODE HERE
validation_datagen = ImageDataGenerator(rescale=1.0/255.) #YOUR CODE HERE
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, #YOUR CODE HERE
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


Note: You can ignore the `UserWarning: Possibly corrupt EXIF data.` warnings.

참고: 'UserWarning: EXIF 데이터가 손상될 수 있습니다.`는 경고를 무시해도 됨

In [ ]:
# Note that this may take some time.
# Model 학습
history = model.fit(train_generator,
                              epochs=50,
                              verbose=1,
                              validation_data=validation_generator) # validation score을 확인하기 위해 validation data를 지정
# The expectation here is that the model will train, and that accuracy will be > 95% on both training and validation
# (여기서는 모델이 훈련되고 정확도는 훈련과 검증에서 모두 95% 초과가 예측될 것임.)
# i.e. acc:A1 and val_acc:A2 will be visible, and both A1 and A2 will be > .9
# (즉, acc:A1, val_Acc:A2가 표시되고 A1, A2 모두 >.9가 됨)

Epoch 1/50
 18/225 [=>............................] - ETA: 7:41 - loss: 1.7367 - accuracy: 0.5500

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

225/225 [==============================] - 512s 2s/step - loss: 0.7317 - accuracy: 0.6357 - val_loss: 0.5393 - val_accuracy: 0.7220
Epoch 2/50
225/225 [==============================] - 503s 2s/step - loss: 0.5156 - accuracy: 0.7506 - val_loss: 0.4695 - val_accuracy: 0.7772
Epoch 3/50
225/225 [==============================] - 502s 2s/step - loss: 0.4474 - accuracy: 0.7892 - val_loss: 0.6104 - val_accuracy: 0.6972
Epoch 4/50
225/225 [==============================] - 507s 2s/step - loss: 0.3842 - accuracy: 0.8251 - val_loss: 0.4258 - val_accuracy: 0.8100
Epoch 5/50
156/225 [===================>..........] - ETA: 2:31 - loss: 0.3292 - accuracy: 0.8551

KeyboardInterrupt: ignored

In [ ]:
# model 분석
# train data에서의 accuracy, loss 그리고 validation data에서의 accuracy, loss를 이용하여 모델을 개선
# 이 값을 이용해 현재 overfitting이 일어나고 있는지, underfitting이 일어나고 있는지, 어떠한 클래스를 잘 예측하지 못하는지 등 파악 가능
%matplotlib inline

import matplotlib.image  as mpimg 
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()


# Desired output. Charts with training and validation metrics. No crash :)

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For `all other broswers`, continue with the next code block and ignore the next one after it.

The ones running the `Colab` on `Safari`, comment out the code block below, uncomment the next code block and run it.

**중요 참고:** 일부 호환성 문제로 인해 'Safari' 브라우저에서 이 노트북을 'Colab'으로 실행하는 경우 업로드할 이미지를 선택한 후 다음 코드 블록에 오류가 발생한다. 다른 모든 브라우저에 대해서는 다음 코드 블록을 계속 진행하고 다음 코드 블록은 무시합니다.

'Safari'에서 'Colab'를 실행하는 사람들은 아래의 코드 블록을 주석으로 남겨 다음 코드 블록을 풀고 실행한다.

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing
# ()
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

For those running this `Colab` on `Safari` broswer can upload the images(s) manually. Follow the instructions, uncomment the code block below and run it.

Instructions on how to upload image(s) manually in a Colab:

1. Select the `folder` icon on the left `menu bar`.
2. Click on the `folder with an arrow pointing upwards` named `..`
3. Click on the `folder` named `tmp`.
4. Inside of the `tmp` folder, `create a new folder` called `images`. You'll see the `New folder` option by clicking the `3 vertical dots` menu button next to the `tmp` folder.
5. Inside of the new `images` folder, upload an image(s) of your choice, preferably of either a horse or a human. Drag and drop the images(s) on top of the `images` folder.
6. Uncomment and run the code block below. 


'Safari' 브라우저에 이 'Colab'을 실행 중인 사용자는 수동으로 이미지를 업로드할 수 있다. 지시사항에 따라 아래 코드 블록의 주석을 풀고 실행하라.

Colab에서 수동으로 이미지를 업로드하는 방법:

1. 왼쪽 메뉴 모음에서 '폴더' 아이콘을 선택
2. '..'라는 이름의 '화살표가 위를 향하고 있는 폴더'를 클릭
3. 'tmp'라는 이름의 폴더를 클릭.
4. 'tmp' 폴더 안에는 '이미지'라는 새로운 폴더를 생성. 'tmp' 폴더 옆에 있는 '세로점 3개' 메뉴 버튼을 클릭하면 '새 폴더' 옵션이 나타남.
5. 새로운 '이미지' 폴더 안에 말이나 사람 중 원하는 이미지를 업로드. 이미지를 '이미지' 폴더 위에 끌어다놓기.
6. 아래의 코드 블록을 제거하고 실행.

In [ ]:
# import numpy as np
# from keras.preprocessing import image
# import os

# images = os.listdir("/tmp/images")

# print(images)

# for i in images:
#  print()
#  # predicting images
#  path = '/tmp/images/' + i
#  img = image.load_img(path, target_size=(150, 150))
#  x = image.img_to_array(img)
#  x = np.expand_dims(x, axis=0)

#  images = np.vstack([x])
#  classes = model.predict(images, batch_size=10)
#  print(classes[0])
#  if classes[0]>0.5:
#    print(i + " is a dog")
#  else:
#    print(i + " is a cat")